# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import torch.nn as nn
import numpy as np

# 00 Game Info

In [ ]:
STATE_SIZE = (3,3) #틱택토 보드 크기
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)

# 01 HYPER PARAMS

In [ ]:
# Hyperparameters
N_SIMULATIONS = 100  # 각 행동에 대한 시뮬레이션 횟수

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# 02 Env+State

In [ ]:
import os

# 파일 복사 명령어 실행
os.system('cp "/content/drive/My Drive/Colab Notebooks/공통 environment+state.ipynb" "/content/"')

0

In [ ]:
import nbformat

notebook_path = "/content/공통 environment+state.ipynb"
with open(notebook_path, "r", encoding="utf-8") as f:
    notebook_content = nbformat.read(f, as_version=4)

# 각 코드 셀 출력 및 실행
for cell in notebook_content.cells:
    if cell.cell_type == "code":
        print(f"실행 중인 코드:\n{cell.source}\n{'='*40}")
        exec(cell.source)

실행 중인 코드:
import torch
import torch.nn as nn
import numpy as np
실행 중인 코드:
STATE_SIZE = (3,3)
N_ACTIONS = STATE_SIZE[0]*STATE_SIZE[1]
STATE_DIM = 3 # first player 정보 넣음
BOARD_SHAPE = (STATE_DIM, 3, 3)
실행 중인 코드:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
실행 중인 코드:
class Environment: #틱택토 게임 환경 정의 클래스 / 게임 규칙을 코드로 구현한 것
    def __init__(self):
        self.n = STATE_SIZE[0] #보드 행 또는 열 크기
        self.num_actions = self.n ** 2
        self.action_space = np.arange(self.num_actions)
        self.reward_dict = {'win': 1, 'lose': -1, 'draw': 0} #결과에 따른 보상 정의
        #승리 1점 / 패배 -1점 / 무승부 0점


    def step(self, present_state, action_idx): #현재 상태에서 주어진 행동에 따라 게임 진행 / 행동 결과를 계산
        """
        present_state에 대해 action_idx의 행동에 따라 게임을 한 턴 진행시키고
        next_state, is_done, is_lose를 반환한다.
        """
        #action_idx: 플레이어가 선택한 행동의 인덱스 / 보드 칸
        #현재 상태에서 행동하는 행동을 수행하여 다음 상태를 계산
        next_state = present_state.next(action_idx)
        is_done, is_lose = ne

# 03 AlphaBeta

In [ ]:
class AlphaBetaAgent:
    def __init__(self, depth=3):
        self.depth = depth


    def get_action(self, state): # 현재 상태에서 최적의 행동 선택
        _, best_action = self._alphabeta(state, self.depth, -float('inf'), float('inf'), True)

        if best_action is None:  # 가능한 행동이 없을 경우 예외 처리 통해 랜덤 선택
            legal_actions = np.where(state.get_legal_actions() != 0)[0]
            if len(legal_actions) > 0:
                print("[WARNING] No optimal actions found, choosing a random action.")
                return np.random.choice(legal_actions)  # 가능한 행동 중 랜덤 선택
            else:
                print("[ERROR] No valid actions available. Game might be in an invalid state.")
                return None  # 안전하게 None 반환

        return best_action


    # AlphaBeta 탐색 알고리즘의 재귀 구현
    def _alphabeta(self, state, depth, alpha, beta, is_maximizing_player):
        # 종료 조건: 게임이 끝났거나 탐색 깊이에 도달한 경우
        is_done, is_lose = state.check_done()
        if is_done or depth == 0 or len(np.where(state.get_legal_actions() != 0)[0]) == 0:
            return self._evaluate_state(state), None

        legal_actions = np.where(state.get_legal_actions() != 0)[0]

        if is_maximizing_player:
            max_eval = -float('inf')
            best_action = None
            for action in legal_actions:
                next_state = state.next(action)
                eval_value, _ = self._alphabeta(next_state, depth - 1, alpha, beta, False)

                # 가지치기 수행
                if eval_value >= beta:
                    return eval_value, action  #가지치기 발생 즉시 반환

                if eval_value > max_eval:
                    max_eval = eval_value
                    best_action = action

                alpha = max(alpha, eval_value)

            return max_eval, best_action
        else:
            min_eval = float('inf')
            best_action = None
            for action in legal_actions:
                next_state = state.next(action)
                eval_value, _ = self._alphabeta(next_state, depth - 1, alpha, beta, True)

                # 가지치기 수행
                if eval_value <= alpha:
                    return eval_value, action  # 가지치기 발생 즉시 반환

                if eval_value < min_eval:
                    min_eval = eval_value
                    best_action = action

                beta = min(beta, eval_value)

            return min_eval, best_action


    def _evaluate_state(self, state):  # 상태 평가 함수
       if state.check_done()[0]:  # 종료 상태라면 보상 반환
           return state.get_reward(state)

       # 개선된 평가 기준 (새로운 변수를 만들지 않음)
       return (
           (np.sum(state.state) - np.sum(state.enemy_state)) * 0.5  # 플레이어 돌 개수 차이
           + (np.sum(state.state == 1, axis=0).max() == 2) * 1.0  # 가로 2개 연속 여부
           + (np.sum(state.state == 1, axis=1).max() == 2) * 1.0  # 세로 2개 연속 여부
           + (np.trace(state.state) == 2) * 1.0  # 대각선 (\) 2개 연속 여부
           + (np.trace(np.fliplr(state.state)) == 2) * 1.0  # 대각선 (/) 2개 연속 여부
       )

## Test Code

In [ ]:
# 1. State 초기화
state = State()
print("초기 상태 보드:")
state.render(state)
print("가능한 행동들:", np.where(state.get_legal_actions() != 0)[0])

초기 상태 보드:
가능한 행동들: [0 1 2 3 4 5 6 7 8]


In [ ]:
# 2. AlphaBeta 초기화
alpha_beta = AlphaBetaAgent(depth=3)

In [ ]:
# 3. 최적 행동 확인
best_action = alpha_beta.get_action(state)
print("AlphaBeta 탐색이 선택한 최적 행동:", best_action)

AlphaBeta 탐색이 선택한 최적 행동: 0


In [ ]:
# 4. 특정 상태 테스트
custom_state = State()
custom_state = custom_state.next(3)  # 첫 번째 플레이어가 3번 칸에 놓음
custom_state = custom_state.next(5)  # 두 번째 플레이어가 5번 칸에 놓음
custom_state = custom_state.next(1)  # 첫 번째 플레이어가 1번 칸에 놓음
custom_state = custom_state.next(2)  # 두 번째 플레이어가 2번 칸에 놓음

print("테스트 상태 보드:")
custom_state.render(custom_state)
print("가능한 행동들:", np.where(custom_state.get_legal_actions() != 0)[0])

best_action_test = alpha_beta.get_action(custom_state)
print("테스트 상태에서 AlphaBeta 선택 행동:", best_action_test)

테스트 상태 보드:
가능한 행동들: [0 4 6 7 8]
테스트 상태에서 AlphaBeta 선택 행동: 8
